In [ ]:
#from datasets import load_dataset

: 

In [ ]:
#dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

: 

In [ ]:
import pandas as pd
from transformers import BertTokenizer

: 

In [ ]:
df_yn = pd.read_csv("C:/Users/USER/삼성화재/git_share/hyper-personalized_car_accident_prediction/Dataset/고객별 사고 유무 기본 데이터.csv", encoding = "UTF-8", engine='python')

: 

In [ ]:
df_yn.dtypes

: 

In [ ]:
def remove_decimal_and_convert_to_str(value):
    if pd.isnull(value):
        return 'nan'
    if isinstance(value, float):
        value = int(value) if value.is_integer() else value
    return str(value)

: 

In [ ]:
df_yn['processed'] = df_yn.apply(lambda row: '/'.join(row.apply(remove_decimal_and_convert_to_str)), axis=1)


: 

In [ ]:
# df_yn['processed'] = df_yn.apply(lambda row: '/'.join(row.astype(str)), axis=1)

: 

In [ ]:
df_yn["processed"][1]

: 

In [ ]:
sentences = df_yn['processed'].tolist()

: 

In [ ]:
labels = [int(sentence.split('/')[-1]) for sentence in sentences]


: 

In [ ]:
%pip install torch torchvision torchaudio


: 

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

: 

In [ ]:
%pip install tokenizers

: 

In [ ]:
# 문장 리스트를 텍스트 파일로 저장
with open("sentences.txt", "w", encoding="utf-8") as file:
    for sentence in sentences:
        file.write(sentence + '\n')

: 

In [ ]:
# from tokenizers import BertWordPieceTokenizer

: 

In [ ]:
# 새 WordPiece 토크나이저 초기화
# tokenizer = BertWordPieceTokenizer(clean_text=True, handle_chinese_chars=True)

: 

In [ ]:
# 토크나이저 훈련
# tokenizer.train(
#     files="sentences.txt",
#     vocab_size=30522,
#     min_frequency=2,
#     show_progress=True,
#     special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
#     limit_alphabet=1000,
#     wordpieces_prefix="##"
# )

# # 훈련된 토크나이저 저장
# tokenizer.save_model(".", "bert-wordpiece")

: 

In [ ]:
# from tokenizers import BertWordPieceTokenizer

# # 저장된 토크나이저 모델 로드
# tokenizer = BertWordPieceTokenizer("bert-wordpiece-vocab.txt", lowercase=True)

# # 테스트할 문장
# test_sentences = df_yn["processed"][1]

# # 각 문장을 토큰화하고 결과 출력
# for sentence in test_sentences:
#     output = tokenizer.encode(sentence)
#     print(f"Original: {sentence}")
#     print(f"Tokens: {output.tokens}")
#     print(f"Token IDs: {output.ids}\n")


: 

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


: 

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.backends.cudnn.version())


: 

In [ ]:
# Load the dataset
df_yn = pd.read_csv("C:/Users/USER/삼성화재/git_share/hyper-personalized_car_accident_prediction/Dataset/고객별 사고 유무 기본 데이터.csv", encoding="UTF-8", engine='python')


: 

In [ ]:
# Function to preprocess the data
def remove_decimal_and_convert_to_str(value):
    if pd.isnull(value):
        return 'nan'
    if isinstance(value, float):
        value = int(value) if value.is_integer() else value
    return str(value)

: 

In [ ]:
labels = df_yn.iloc[:, -1].tolist()
df_yn = df_yn.iloc[:, :-1]
df_yn['processed'] = df_yn.apply(lambda row: '/'.join(row.apply(remove_decimal_and_convert_to_str)), axis=1)


: 

In [ ]:
# Extract sentences
sentences = df_yn['processed'].tolist()

: 

In [ ]:
toy_sentences, eval_sentences, toy_labels, eval_labels = train_test_split(sentences, labels, test_size=0.5, random_state=42)

: 

In [ ]:
# Split data into train and test sets
train_sentences, test_sentences, train_labels, test_labels = train_test_split(toy_sentences, toy_labels, test_size=0.2, random_state=42)

: 

In [ ]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

: 

In [ ]:
# Tokenize the sentences
train_encodings = tokenizer(train_sentences, padding=True, truncation=True, return_tensors="pt")
test_encodings = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="pt")


: 

In [ ]:
# Create a Dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

: 

In [ ]:
# Create datasets
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

: 

In [ ]:
# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

: 

In [ ]:
# Load BERT model for binary classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

: 

In [ ]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


: 

In [ ]:
# Training the model
model.train()
for epoch in range(3):  # Train for 3 epochs
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

: 

In [ ]:
# Save the model
model_save_path = "bert_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

: 

In [ ]:
# Evaluate the model
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).tolist())
        true_labels.extend(labels.tolist())

: 

In [ ]:
# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

: 

In [ ]:
# Load the model (example usage)
model_load_path = "bert_model.pth"
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_state_dict(torch.load(model_load_path))
model.to(device)
print(f"Model loaded from {model_load_path}")

: 